In [14]:
import zarr
import xarray as xr
import numpy as np
from boto3 import client as boto_client 

In [15]:
chunk = 262144
to_rad = np.pi / 180
to_deg = 180 / np.pi

campaign = 'Olympex'
collection = "AirborneRadar"
dataset = "gpmValidationOlympexcrs"
variables = ["ref"]
renderers = ["point_cloud"]

In [16]:
def down_vector(roll, pitch, head):
    x = np.sin(roll) * np.cos(head) + np.cos(roll) * np.sin(pitch) * np.sin(head)
    y = -np.sin(roll) * np.sin(head) + np.cos(roll) * np.sin(pitch) * np.cos(head)
    z = -np.cos(roll) * np.cos(pitch)
    return (x, y, z)

def CRSaccess(fname, s3bucket=False, Verb=False):
    """
    Access the CRS file
    Return CRS filename with path (absolute path) for "local" access
    Return CRS data as object for "cloud access"
    Either way, the return value can be open by Xarray as netcdf file object
    """

    print("\%% Accessing data from Cloud. This may take a little time...\n")
    s3 = boto_client('s3')
    fileobj = s3.get_object(Bucket=s3bucket, Key=fname)
    fileCRS = fileobj['Body'].read()

    return fileCRS

def add24hr(hr):
    """Correction of time in CRS for going over the next day in UTC"""
    b = np.where(hr < hr[0])
    hr[b] = hr[b] + 24
    return hr



In [17]:
def ingest(folder, file, s3bucket):
    """
    Converts Level 1B crs data from s3 to zarr file and then stores it in the provided folder
    Args:
        folder (string): name to hold the raw files.
        file (string): the s3 url to the raw file.
    """
    store = zarr.DirectoryStore(folder)
    root = zarr.group(store=store)
    
    # !!! Define dataset for necessary generic data.
    z_chunk_id = root.create_dataset('chunk_id', shape=(0, 2), chunks=None, dtype=np.int64)
    z_location = root.create_dataset('location', shape=(0, 3), chunks=(chunk, None), dtype=np.float32)
    z_time = root.create_dataset('time', shape=(0), chunks=(chunk), dtype=np.int32)

    # !!! Define group for value
    z_vars = root.create_group('value')
    # !!! Define dataset for actual value that will get plotted.
    z_ref = z_vars.create_dataset('ref', shape=(0), chunks=(chunk), dtype=np.float32)

    # !!! define a empty array. FOR WHAT??? DONOT HAVE ANY IDEA.
    n_time = np.array([], dtype=np.int64)

    # !!! Extract the date information from the file name. The date is the date when the data was collected from the instrument.
    date = file.split("_")[2]
    base_time = np.datetime64('{}-{}-{}'.format(date[:4], date[4:6], date[6:]))

    print("Accessing file from S3 ", file)

    # read from s3 url (file) in s3 bucket.
    fileObj = CRSaccess(file, s3bucket=s3bucket)

    # open dataset.
    with xr.open_dataset(fileObj, decode_cf=False) as ds:
        #### time correction start
        # !!! (logic????) added for time correction for time (hour) over 24h UTC
        hr = add24hr(ds['timed'].values)

        # !!! addition of date to all time rows i.e. {date + time} for all rows.
        # Note: "delta" is an intermediatery before final time correction.
        delta = (hr * 3600).astype('timedelta64[s]') + base_time
        # now delta will have this sort of value: ['2015-11-10T17:28:16' '2015-11-10T17:28:16' '2015-11-10T17:28:17' ...
        # '2015-11-10T18:01:05' '2015-11-10T18:01:05' '2015-11-10T18:01:05']
        
        #### time correction end
        
        # Data COLS EXTRACT 
        ref = ds["zku"].values #CRS radar reflectivity
        rad_range = ds["range"].values

        lat = ds['lat'].values
        lon = ds['lon'].values
        alt = ds['altitude'].values # altitude of aircraft in meters
        roll = ds["roll"].values
        pitch = ds["pitch"].values
        head = ds["head"].values
    num_col = ref.shape[0] # number of cols, say 7903
    num_row = ref.shape[1] # number of rows, say 757

    ##### EXTRACTED COLS
    # timed, lat, lon, alt, roll, pitch, head,
    # ref(zku), rad_range(range)
    #####

    ##### data frame formation

    ## repeat each value element consecutively row times. WHY???
    delta = np.repeat(delta, num_row)
    lon = np.repeat(lon, num_row)
    lat = np.repeat(lat, num_row)
    alt = np.repeat(alt, num_row)
    roll = np.repeat(roll * to_rad, num_row)
    pitch = np.repeat(pitch * to_rad, num_row)
    head = np.repeat(head * to_rad, num_row)

    ## repeat each value element consecutively col times
    rad_range = np.tile(rad_range, num_col)
    ## !!! REPEAT OTHER DATA WRT REF (THE MAIN THING TO PLOT), THEN FLATTEN THE REF.
    ref = ref.flatten()

    # time correction final
    # !!! before subtraction, first the datetime is set to precision of seconds, then it is converted into int64
    time = (delta - np.datetime64('1970-01-01')).astype('timedelta64[s]').astype(np.int64)

    # !!! needed for {lon lat alt} correction with respect to rad range and roll, pitch, head value. Why is it necessary ???
    x, y, z = down_vector(roll, pitch, head)
    x = np.multiply(x, np.divide(rad_range, 111000 * np.cos(lat * to_rad)))
    y = np.multiply(y, np.divide(rad_range, 111000))
    z = np.multiply(z, rad_range)
    lon = np.add(-x, lon)
    lat = np.add(-y, lat)
    alt = np.add(z, alt)

    # !!! sort data by time
    sort_idx = np.argsort(time) # returns indices that will sort the array
    lon = lon[sort_idx]
    lat = lat[sort_idx]
    alt = alt[sort_idx]
    ref = ref[sort_idx]
    time = time[sort_idx]

    # !!! Remove nan and infinite using mask
    # check if ref is finite and alt value is greater than 1, element wise
    # index mask to filter infinite ref value and alt value which is greater than 1
    mask = np.logical_and(np.isfinite(ref), alt > 0)
    lon = lon[mask]
    lat = lat[mask]
    alt = alt[mask]
    ref = ref[mask]
    time = time[mask]

    ### !!! ZARR DATASETS POPULATION
    # Now populate (append) the empty rows with modified data.
    z_location.append(np.stack([lon, lat, alt], axis=-1)) #!!! first convert position info as [[lon, lat, alt], ....], before appending
    z_ref.append(ref)
    n_time = np.append(n_time, time) # Basically converts into array with single dim; very similar to spread operator.

    idx = np.arange(0, n_time.size, chunk) # arange: Return evenly spaced values within a given interval. (start, stop, steps)
    chunks = np.zeros(shape=(idx.size, 2), dtype=np.int64) # create zero array of given shape. here (indexsize x 2)
    chunks[:, 0] = idx
    chunks[:, 1] = n_time[idx]
    z_chunk_id.append(chunks)
    
    epoch = np.min(n_time)
    n_time = (n_time - epoch).astype(np.int32) ## why done this??? i.e. adding date to time information first, then later subtract datetime. 
    z_time.append(n_time)

    # save it.
    root.attrs.put({
        "campaign": campaign,
        "collection": collection,
        "dataset": dataset,
        "variables": variables,
        "renderers": renderers,
        "epoch": int(epoch)
    })

In [18]:
folder = f"/tmp/crs_olympex/zarr/11111111"
bucket_name="ghrc-fcx-field-campaigns-szg"
s3_raw_file_key= "Olympex/instrument-raw-data/crs/olympex_CRS_20151110_172815-20151110_175946_2_v01a.nc"

ingest(folder, s3_raw_file_key, bucket_name)

Accessing file from S3  Olympex/instrument-raw-data/crs/olympex_CRS_20151110_172815-20151110_175946_2_v01a.nc
\%% Accessing data from Cloud. This may take a little time...



Read the written file

In [20]:
store = zarr.DirectoryStore(folder)
root = zarr.group(store=store)
root.tree()

AttributeError: 'Tree' object has no attribute '_ipython_display_'

/
 ├── chunk_id (3, 2) int64
 ├── location (646799, 3) float32
 ├── time (646799,) int32
 └── value
     └── ref (646799,) float32

### ROUGH

In [ ]:
np.array([], dtype=np.int64)

In [ ]:
date="20151009"
dt = np.datetime64('{}-{}-{}'.format(date[:4], date[4:6], date[6:]))
dt

In [ ]:
dt1 = np.datetime64('2022-01-01').astype('timedelta64[s]')
print(dt1)
dt2 = dt1.astype(np.int64)
print(dt2)

In [ ]:
ttt = ['2015-11-10T17:28:16', '2015-11-10T17:28:16', '2015-11-10T17:28:17', '2015-11-10T18:01:05', '2015-11-10T18:01:05', '2015-11-10T18:01:05']
np.repeat(ttt, 2)

In [ ]:
lon = np.array([1, 2, 3])
lat = np.array([4, 5, 6])
np.stack(([lon, lat]), axis=-1)

In [ ]:
np.append([1, 2, 3], [[4, 5, 6], [7, 8, 9]])

In [ ]:
tmw = ([np.datetime64(x) for x in ttt]  - np.datetime64('1970-01-01')).astype('timedelta64[s]').astype(np.int64)
tmw

In [ ]:
np.min(tmw)